In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

In [18]:
data = np.arange(100)
probs = [0.15] * 3
data, probs

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 [0.15, 0.15, 0.15])

In [20]:
parts = list(map(int, len(data) * np.array([*probs])))
idx = torch.randperm(len(data)).numpy()
res = np.split(data[idx], parts)
print(f"{len(res)=}\n{res=}")

len(res)=4
res=[array([ 0, 58, 60, 57, 24, 97, 44, 59,  1, 79, 72, 81, 23,  7, 37]), array([], dtype=int64), array([], dtype=int64), array([42, 48,  8, 98, 84,  6, 73, 47, 52, 28, 55, 41, 25, 95, 51, 43, 88,
       99, 40, 18, 78, 80, 67,  2, 61, 75, 33, 70, 39, 64, 85, 92, 69, 71,
       14, 94, 31, 90,  3, 45, 19,  5, 66, 21,  4, 65, 15, 76, 17, 46, 11,
       34, 87, 29, 93, 10, 63, 74, 62, 91, 56, 50, 13,  9, 89, 53, 26, 32,
       36, 20, 68, 35, 12, 27, 83, 82, 49, 77, 38, 54, 86, 96, 16, 22, 30])]


In [ ]:
import json
import importlib

import Code.WandbRunner as WandbRunnerLib
importlib.reload(WandbRunnerLib)
import Code.WandbRunner as WandbRunnerLib

with open('configs/config_1.json') as file:
    config = json.loads(file.read())
    
    WandbRunnerLib.CreateRun(config)

In [21]:
import json
import importlib

import Code.WandbRunner as WandbRunnerLib
importlib.reload(WandbRunnerLib)
import Code.WandbRunner as WandbRunnerLib

with open('configs/config_1.json') as file:
    config = json.loads(file.read())
    
    WandbRunnerLib.CreateRun(config)

Epoch [79/100]
Train Loss: 0.2521
Validation Loss: 0.2508
lr: [0.00012288000000000002]


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

epoch,▁▃▆█
std,▁
std_sq,▁
train_loss,█▃▂▁
val_loss,█▅▂▁
epoch,79
std,0.5
std_sq,0.25
train_loss,0.25215
val_loss,0.25076


In [23]:
import json
import importlib

import Code.WandbRunner as WandbRunnerLib
importlib.reload(WandbRunnerLib)
import Code.WandbRunner as WandbRunnerLib

with open('configs/config_3.json') as file:
    config = json.loads(file.read())
    
    WandbRunnerLib.CreateRun(config)

Epoch [296/300]
Train Loss: 1.3170
Validation Loss: 1.3567
lr: [0.0003]


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
std,▁
std_sq,▁
train_loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,██▇▇▇▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch,296
std,0.91904
std_sq,0.84463
train_loss,1.31703
val_loss,1.35668


# Обучим gpt2 на данных

In [1]:
from transformers import GPT2Tokenizer, TFGPT2Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4,5,6,7"

In [3]:
device = 'cpu' if not torch.cuda.is_available() else 'cuda:3'
device

'cuda:3'

# Testing crio
 

In [3]:
import pandas as pd
import numpy as np
import json
import typing as tp
from tqdm.notebook import tqdm
from plotly_resampler import FigureResampler

In [4]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

In [5]:
import Code.Dataloader as DataloaderLib
import Code.Logger as LogsLib
import Code.Dataset as DatasetLib
import Code.ModelBertV1 as ModelLib
import Code.Train as TrainLib

%load_ext autoreload
%autoreload 2

import importlib
importlib.reload(DataloaderLib)
importlib.reload(DatasetLib)
importlib.reload(LogsLib)
importlib.reload(ModelLib)
importlib.reload(TrainLib)

import Code.Dataloader as DataloaderLib
import Code.Logger as LogsLib
import Code.Dataset as DatasetLib
import Code.ModelBertV1 as ModelLib
import Code.Train as TrainLib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# start_ind = int(1e7)
# transactions = DataloaderLib.GetTransactionsByInd(range(start_ind, start_ind + 150), logger_fake='logger_loader')

In [2]:
import Code.Dataloader as loader

In [3]:
import Code.Dataloader as loader
fetcher = loader.SyncBatchEthFetcher(batch_size=500)
last_ind = int(2e7 + 2e6)
size = 100_000

df_txs = fetcher.get_transactions(last_ind - size, last_ind, use_async=True)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [95]:
df_txs.timestamp.max() - df_txs.timestamp.min()

Timedelta('13 days 16:28:12')

In [96]:
df_txs

(2569451, 4)

In [61]:
# df_to_for_predict.value[df_to_for_predict.value > 0].min()

In [56]:
# last_ind = int(2e7 + 1e6)
# size = 1000
transactions = DataloaderLib.GetTransactionsByInd(range(last_ind, last_ind - size, -1), logger_fake='logger_loader')
df_to_for_predict = transactions.loc[:, DataloaderLib.useful_columns]

  0%|          | 0/40 [00:00<?, ?it/s]

Write at logger: logs/logger_loader.log


In [27]:
df_to_for_predict

,timestamp,from,to,value
0,2024-10-19 13:45:47,1153599334270540825116534873609702723577695361442,360626852303502211918984836330587137786732780243,0.000000
1,2024-10-19 13:45:47,97840791646144403883154721377920130019782939081,198711195697986279793524651792786476427874121482,0.000000
2,2024-10-19 13:45:47,1251277645027622750093042385120322446303808392872,331497450208276499761800402457369371151716489247,0.300000
3,2024-10-19 13:45:47,280854366049012869853841418439870510546273224552,466869609986913014237362913113971488315126327935,0.000000
4,2024-10-19 13:45:47,1228251626684428057107246147496081400699385431934,292072159098574616991920578893626132055744093257,0.000000
...,...,...,...,...
177713,2024-10-19 10:24:59,490928619411245153474035081455369690121131017400,917551056842671309452305380979543736893630245704,0.000000
177714,2024-10-19 10:24:59,7984248247461868079348554290722630341108976110,1248875146012964071876423320777688075155124985543,0.000000
177715,2024-10-19 10:24:59,1056152795177899898594824636591353623429882988491,515704586464927703177239230318634714919195578546,0.000000
177716,2024-10-19 10:24:59,1131381527668650895112316810790933854470020142151,1300486112865620505796167037738822718394911465103,0.000000


In [9]:
# last_ind = int(2e7 + 1e6)
# size = 4000
# transactions = DataloaderLib.GetTransactionsByInd(range(last_ind, last_ind - size, -1), logger_fake='logger_loader')
# df_to_for_predict = transactions.loc[:, DataloaderLib.useful_columns]

In [10]:
# last_ind = int(2e7 + 1e6)
# size = 6000
# transactions = DataloaderLib.GetTransactionsByInd(range(last_ind, last_ind - size, -1), logger_fake='logger_loader')
# dataset = transactions.loc[:, DataloaderLib.useful_columns]

In [40]:
# # %%time
# # cnt_trades = 500_000
# cnt_trades = 500_000
# # cnt_trades = 30
# # df_raw = DataloaderLib.DatasetTimeBoomAuthorDependence(cnt_trades, count_people=20, cnt_boom=cnt_trades // 100, sigma=10000)
# df_raw = DataloaderLib.DatasetTimeBoomAuthorDependence(cnt_trades, count_people=20, cnt_boom=0, sigma=10000)
# # df_raw = DataloaderLib.SimpleDataloader(cnt_trades, count_people=10,
# #                                         volume_strategy=lambda size: np.ones((size)) * 0.5,
# #                                         volume_strategy_param={},
# #                                         )
# # df_raw = DataloaderLib.SimpleDataloader(cnt_trades, count_people=5)


Write at logger: logs/logs_149.log


In [41]:
df_raw

,from,to,value,timestamp
0,15,0,5.63138,2.924401
1,7,17,6.270023,5.052833
2,12,7,5.371346,7.450858
3,0,16,4.600258,9.899299
4,15,6,4.037146,10.907945
...,...,...,...,...
499995,12,14,4.617317,999813.000278
499996,0,19,3.871414,999815.428216
499997,3,8,3.429647,999817.783648
499998,12,8,4.187352,999819.16025


In [43]:
def DrawTransactionDF(df, bound_show=1e4):
    assert set(df.columns) == set(['from', 'to', 'value', 'timestamp'])
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['value'],
        line_shape='hv',
    ))
    resampler = FigureResampler if len(df) > bound_show else lambda x: x
    return resampler(fig)

DrawTransactionDF(df_raw.iloc[:int(3e4)], bound_show=1e5)

In [44]:
(df_raw.value - df_raw.value.mean()).std(), (df_raw.value - df_raw.value.mean()).std() ** 2

(np.float64(0.9217822865831987), np.float64(0.8496825838585503))

In [45]:
df_to_for_predict = df_raw

In [46]:
train_dataset, val_dataset = train_test_split(df_raw, test_size=1/4, shuffle=False)

In [47]:
train_dataset.shape, val_dataset.shape

((375000, 4), (125000, 4))

# Смотрим на датасет

In [48]:
df_to_for_predict

,from,to,value,timestamp
0,15,0,5.63138,2.924401
1,7,17,6.270023,5.052833
2,12,7,5.371346,7.450858
3,0,16,4.600258,9.899299
4,15,6,4.037146,10.907945
...,...,...,...,...
499995,12,14,4.617317,999813.000278
499996,0,19,3.871414,999815.428216
499997,3,8,3.429647,999817.783648
499998,12,8,4.187352,999819.16025


In [49]:
df_to_for_predict.head(5)

,from,to,value,timestamp
0,15,0,5.63138,2.924401
1,7,17,6.270023,5.052833
2,12,7,5.371346,7.450858
3,0,16,4.600258,9.899299
4,15,6,4.037146,10.907945


In [50]:
len(df_to_for_predict['from'].value_counts())

20

# Bert!!!

In [51]:
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertModel
import numpy as np

In [52]:
cnt_known_address = int(1e3)
known_address = pd.concat([df_to_for_predict['to'], df_to_for_predict['from']]).value_counts()[:cnt_known_address].keys().sort_values()

In [53]:
known_address

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype='object')

In [54]:
df_raw

,from,to,value,timestamp
0,15,0,5.63138,2.924401
1,7,17,6.270023,5.052833
2,12,7,5.371346,7.450858
3,0,16,4.600258,9.899299
4,15,6,4.037146,10.907945
...,...,...,...,...
499995,12,14,4.617317,999813.000278
499996,0,19,3.871414,999815.428216
499997,3,8,3.429647,999817.783648
499998,12,8,4.187352,999819.16025


In [55]:
dataset = DatasetLib.TransactionDataset(df_raw, list(known_address), 100)
for name, value in dataset[0].items():
    print(f"{name}: {value.shape}")

numeric_features: torch.Size([100, 1])
from_address: torch.Size([100])
to_address: torch.Size([100])
time_features: torch.Size([100, 3])
value: torch.Size([100])


In [56]:
dataset_params = dict(
    known_address = list(known_address),
    sample_len = 100,
)

In [58]:
train_dataloader = torch.utils.data.DataLoader(
    DatasetLib.TransactionDataset(train_dataset, **dataset_params),
    batch_size=64,
    shuffle=True,
    num_workers=0)

val_dataloader = torch.utils.data.DataLoader(
    DatasetLib.TransactionDataset(val_dataset, **dataset_params),
    batch_size=64,
    shuffle=True,
    num_workers=0)

# Train with time multiplying 

In [59]:
model = ModelLib.BertForTransactionRegressionV1(len(known_address), 4).to(device)


In [61]:
train_params = dict(
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    time_coef_loss=1,

    cnt_last_for_show=50,
    seconds_betwen_image_show=10,

    start_epoch=0,
    num_epochs=30,
    learning_rate=3e-4,
    gamma=0.8,

    step_size=10,
    device=device,
)

In [62]:
# model = nn.DataParallel(model).to("cuda")
TrainLib.train_model(model=model, model_predictor=TrainLib.time_cross_predictior,
                     result_loss=TrainLib.result_loss_slower_change, **train_params)

Epoch [30/30]
Train Loss: 1.2577
Test Loss: 1.1910
lr: [0.0003]


In [52]:
model = ModelLib.BertForTransactionRegressionV1(len(known_address), 4).to(device)
# model = nn.DataParallel(model).to("cuda")
TrainLib.train_model(model=model, model_predictor=TrainLib.time_cross_predictior,
                     result_loss=TrainLib.result_loss_slower_change, **train_params)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Train by cls multiplying

In [97]:
model = ModelLib.BertForTransactionRegressionV1(len(known_address), 4).to(device)
TrainLib.train_model(model=model, model_predictor=TrainLib.scalar_dot_predictor, **train_params)

Epoch [6/500]
Train Loss: 28.9666
Test Loss: 28.0238
lr: [0.0003]


  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

# Train with 3 dimention tensor

In [39]:
device = 'cuda:3'
# device = 'cpu'

In [37]:
model = ModelLib.BertForTransactionRegressionV1(len(known_address), 64, use_compositor=True).to(device)

In [38]:
TrainLib.train_model(
    model=model,
    model_predictor=TrainLib.time_cross_predictior,
    result_loss=TrainLib.result_loss_slower_change,
    # train_loader=train_dataloader,
    # val_loader=val_dataloader,
    # num_epochs=1000,
    # learning_rate=3e-4,
    # start_epoch=0,
    # gamma=0.8,
    # step_size=10,
    # device=device,
    # time_coef_loss=1,
    # seconds_betwen_image_show=30,
    # cnt_last_for_show=20
    **train_params
)

Epoch [30/30]
Train Loss: 0.2603
Test Loss: 0.2583
lr: [0.0003]


In [45]:
batch = next(iter(val_dataloader))

In [47]:
p_change, p_msk = 0.15, 0.15

In [50]:
change_strategy = TrainLib.uniform_change_strategy
model_predictor = TrainLib.time_cross_predictior
loss_fn = TrainLib.criterion_loss_fn
criterion = nn.MSELoss()
result_loss=TrainLib.result_loss_slower_change
time_coef_loss=1

In [52]:
volumes_pred

tensor([[0.5112, 0.4521, 0.4885,  ..., 0.5029, 0.5281, 0.5073],
        [0.5243, 0.4954, 0.5009,  ..., 0.5041, 0.4507, 0.4840],
        [0.4866, 0.4988, 0.5044,  ..., 0.4976, 0.4866, 0.4979],
        ...,
        [0.4800, 0.4229, 0.4434,  ..., 0.4773, 0.4488, 0.4656],
        [0.4876, 0.5021, 0.5246,  ..., 0.4996, 0.4831, 0.5115],
        [0.5050, 0.5033, 0.4875,  ..., 0.5229, 0.4964, 0.5049]],
       device='cuda:3', grad_fn=<ViewBackward0>)

In [63]:
batch['value'].to(device)

tensor([[0., 1., 0.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        ...,
        [0., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 0.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 0., 1.]], device='cuda:3')

In [58]:
result.shape

torch.Size([32, 100, 64])

In [59]:
volumes_pred.shape

torch.Size([32, 100])

In [61]:
((batch['value'].to(device) - volumes_pred) ** 2).mean()

tensor(0.2506, device='cuda:3', grad_fn=<MeanBackward0>)

In [51]:
batch_size, cnt_words = batch['to_address'].shape

msk_or_change = np.random.choice(a=range(cnt_words), size=int(cnt_words * (p_change + p_msk)), replace=False)
msk_ind = np.random.choice(a=msk_or_change, size=int(cnt_words * p_msk))
change_ind = np.setdiff1d(msk_or_change, msk_ind)
cnt_change = len(change_ind)

volumes_features = batch['numeric_features']
volumes_features[:, change_ind] = torch.unsqueeze(change_strategy((batch_size, cnt_change)), 2)

res = model(
    numeric_features=volumes_features.to(device),
    from_address=batch['from_address'].to(device),
    to_address=batch['to_address'].to(device),
    time_features=batch['time_features'].to(device),
    msk_ind=msk_ind + 1
)
cls_result = res['cls_result']
result = res['result']
from_emb = res['from_emb']
to_emb = res['to_emb']

volumes_pred = model_predictor(
    cls_result=cls_result,
    result=result,
    from_emb=from_emb,
    to_emb=to_emb,
    use_compositor=model.use_compositor,
    model=model,)
loss_fn(
    pred=volumes_pred,
    target=batch['value'].to(device),
    msk_ind=msk_ind,
    change_ind=change_ind,
    criterion=criterion,).cpu().detach().item(), \
        result_loss(
    result=result, coef=time_coef_loss).cpu().detach().item()

(0.2504632771015167, 0.04760555177927017)

# Adding wandb

In [68]:
import wandb

run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    # entity="my-awesome-team-name",
    # Set the wandb project where this run will be logged.
    project="Diploma",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
    name='run_1'
)

run.log(dict(msg=2, r2=1, corr=-1))

# Задачи

## Model

- задать веса для эмбедингов внутри модели
- написать модель предсказывающую временной эмбединг, который должен не сильно меняться
- поиграть с размером окна/разбитием на окна по метапараметрам (воле/чему-то ещё)
- попробовать модель = один слой атеншена


## Данные

- наладить процесс скачивания
- узнать таки, за какие даты у меня данные 
- а что с генерацией можно ещё придумать???
- нагенерировать идей для генерации